<a href="https://colab.research.google.com/github/Ana-AlonsoCanizares/AA_GRUPO3/blob/main/Limpieza_SAAF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Limpieza datos SAAF


##Descarga y limpeza de datos

Comenzamos cargando los datos de la carpeta de Google Drive en la que estén guardados (será necesario cambiar esta ruta en función de la ruta del ordenador).

Es necesario aceptar la conexión con la cuenta de Google Drive.

Importante y esencial, antes de tirar el código es necesario que la carpeta de Drive a la que se va a llamar tenga los archivos en formato xlsx así nombrados: 'CU_SAAF.xlsx', 'CU_AT.xlsx', 'CU_EMPLEO.xlsx', 'CU_LOG.xlsx' para que no haya errores a la hora de ejecutar este código.

**¡OJO! Primeros archivo con extensión xlsx y los de las zonas a clasificar, con extensión csv, en teoría, los csv no deberían modificarse (a menos que las normativas cambien y con ello las zonas de impacto se modifiquen).**

**Y todos las columnas deben tener los nombres definidos en el documento "Resumen de datos por servicio FASPAS" y en mayúsculas antes de ser subidos al programa.**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Se importan las librerías necesarias para la parte de carga de datos y su limpieza.

In [ ]:
pip install unidecode

In [ ]:
import os
import pandas as pd
import numpy as np
from unidecode import unidecode
import matplotlib.pyplot as plt

In [ ]:
# Asegúrarse de cambiar la ruta por el nombre real de la carpeta en Google Drive
folder_path = '/content/drive/My Drive/PFG_FASPAS/SAAF'
docs_xlsx = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
print(docs_xlsx)

# # Asegúrarse de cambiar la ruta por el nombre real de la carpeta en Google Drive
# folder_path_csv = '/content/drive/My Drive/PFG_FASPAS/Zonas_Prioritarias'
# docs_csv = [f for f in os.listdir(folder_path_csv) if f.endswith('.csv')]
# print(docs_csv)

['TO_SAAF.xlsx', 'CU_SAAF.xlsx', 'AB_SAAF.xlsx', 'GU_SAAF.xlsx', 'CR_SAAF.xlsx']
['municipios_prioritarios_clm.csv']


Creación de un diccionario que almacene los dataframes (.xlsx) contenidos en la carpeta de Cuenca.

Se irán cargando todos los archivos en un dataframe que se añade al diccionario con el nombre de dicho archivo como clave.

In [ ]:
dic_dataframes = {}
dic_zonas = {}

for doc in docs_xlsx:
    entire_path = os.path.join(folder_path, doc)
    df = pd.read_excel(entire_path)
    # Uso el nombre del archivo como clave
    dic_dataframes[doc] = df

# for doc in docs_csv:
#   entire_path = os.path.join(folder_path_csv, doc)
#   df = pd.read_csv(entire_path)
#   # Uso el nombre del archivo como clave
#   dic_zonas[doc] = df

A continuación se debe observar que el dataframe cuente con al menos las columnas: 'AÑOS ATENCIÓN', 'FECHA NACIMIENTO' **en formato fecha**, 'LOCALIDAD', 'GÉNERO' y 'CP'

Ahora que sabemos cuáles son las claves, guardamos cada dataframe por separado para trabajar con todos ellos de manera individual, por el momento.

In [ ]:
# Imprimir todas las claves
for clave in dic_dataframes.keys():
    print(clave)

TO_SAAF.xlsx
CU_SAAF.xlsx
AB_SAAF.xlsx
GU_SAAF.xlsx
CR_SAAF.xlsx


Vamos a limpiar cada uno de los dataframes en función de los datos que tenemos y vamos a normalizarlos para cuando los crucemos entre ellos.

Comenzamos con los datos de Servicio de Atención y Apoyo a Familias (SAAF).

Sacamos la información del dataset y observamos que de 77 entradas, el campo de 'GRADO DEPENDENCIA' tan solo tiene 1 registro, por lo que se procede a eliminarla. El 'GRADO DISCAPACIDAD' de momento lo dejamos ya que para el impacto territorial no lo vamos a usar.

In [ ]:
# # CÓDIGO SOLO PARA EL CURRO (PQ NO PUEDO ACCEDER AL DRIVE)
# CU_SAAF = pd.read_excel('/content/CU_SAAF.xlsx')
# TO_SAAF = pd.read_excel('/content/TO_SAAF.xlsx')
# AB_SAAF = pd.read_excel('/content/AB_SAAF.xlsx')
# GU_SAAF = pd.read_excel('/content/GU_SAAF.xlsx')
# CR_SAAF = pd.read_excel('/content/CR_SAAF.xlsx')

Primero se ponen todos los nombres de las columnas y todos los valores de los registros en mayúsculas y se eliminan todas las tildes contenidas (tanto en los registros de tipo texto como en las columnas), para uniformar el dataset.

In [ ]:
# Definir la función que realiza las transformaciones
def transform_df(df):
    col_excluded = ['FECHA NACIMIENTO', 'FECHA DE NACIMIENTO', 'FECHA NAC']

    # Convertir todas las columnas de tipo object a mayúsculas, excepto las especificadas
    for column in df.columns:
        if df[column].dtype == 'object' and column not in col_excluded:
            df[column] = df[column].apply(lambda x: unidecode(x.upper()) if isinstance(x, str) else x)

    # Ahora, aplicamos unidecode a los nombres de las columnas
    df.columns = [unidecode(col.upper()) for col in df.columns]

    return df

# Asegurarse de que todas las columnas necesarias existan en todos los dataframes, creándolas si es necesario (OJO: COLUMNAS NECESARIAS PARA TODOS LOS SERVICIOS)
# Limpieza de todas aquellas columnas no pedidas
def kept_columns(df):
  # Lista de columnas necesarias para hacer el impacto territorial en todos los dataframes
  all_needed_columns = ['FECHA NACIMIENTO', 'LOCALIDAD', 'GENERO', 'CP']
  for col in all_needed_columns:
    if col not in df.columns:
      df[col] = np.nan

  columns_to_keep = ['FECHA ALTA','FECHA DE ALTA', 'FECHA NACIMIENTO','FECHA DE NACIMIENTO','FECHA NAC' ,'GENERO', 'TIPO USUARIO','LOCALIDAD', 'CP', 'PROVINCIA','TIPO SORDERA', 'MOMENTO APARICION', '?LLEVA AUDIFONO?', '?LLEVA I.C.?', 'AUDIFONO','?LLEVA IMPLANTE DE TRONCO CEREBRAL?', '?LLEVA IMPLANTE OSTEOINTEGRADO?', '?LLEVA IMPLANTE DE OIDO MEDIO?', 'MODALIDAD COMUNICATIVA', 'MODALIDAD COMUNICATIVA EN CASA','MOD COM','GRADO DISCAPACIDAD', 'TIPO PROTESIS', 'TIPO DE PROTESIS','TIPO DE PROTESIS AUDITIVA', 'GRADO PERDIDA', 'GRADO DE PERDIDA']
  df = df[df.columns.intersection(columns_to_keep)]
  return df

def mapping_names(df):
    # Diccionario con los mapeos deseados
    columns_map = {
        'FECHA DE ALTA': 'FECHA ALTA',
        'FECHA NAC': 'FECHA NACIMIENTO',
        'FECHA DE NACIMIENTO': 'FECHA NACIMIENTO',
        'MOMENTO DE APARICION DE LA SORDERA': 'MOMENTO APARICION',
        'TIPO DE PROTESIS AUDITIVA': 'TIPO PROTESIS',
        'TIPO DE PROTESIS': 'TIPO PROTESIS',
        'TIPO DE SORDERA': 'TIPO SORDERA',
        'TIPO DE USUARIO DEL SAAF': 'TIPO USUARIO',
        'MOD COM': 'MODALIDAD COMUNICATIVA',
        'MODALIDAD COMUNICATIVA EN CASA': 'MODALIDAD COMUNICATIVA',
        'GRADO DE PERDIDA': 'GRADO PERDIDA'
    }

    # Crear un nuevo diccionario para los nombres de columnas
    rename_columns = {}

    # Iterar sobre las columnas y aplicar el mapeo
    for col in df.columns:
        norm_col = columns_map.get(col, col)
        rename_columns[col] = norm_col

    # Renombrar las columnas del DataFrame
    df.rename(columns=rename_columns, inplace=True)
    return df

def CP_year(df):

  df['FECHA NACIMIENTO'] = df['FECHA NACIMIENTO'].apply(lambda x: np.nan if str(x).isdigit() or x == '(ADULTO)' else x)
  # df['FECHA NACIMIENTO'] = pd.to_datetime(df['FECHA NACIMIENTO'], errors='coerce')

  df['CP'] =df['CP'].astype('Int64')

  df['YEAR NACIMIENTO'] = df['FECHA NACIMIENTO'].dt.year

  df = df.dropna(subset=['CP', 'LOCALIDAD'], how='all')

  CP_loc_filter = df['CP'].isnull() & df['LOCALIDAD'].isnull()

  df = df[~CP_loc_filter]
  return df

def delete_rows(df):
  df = df.dropna(subset=['CP', 'LOCALIDAD'], how='all')
  return df

Vamos a comenzar a limpiar los datos del Servicio de Asistencia y Apoyo a Familias (sAAF).

Lo primero que hacemos es eliminar aquellas columnas cuyos datos NO se hayan pedido, ya que aunque no se hayan podido recopilar todos los datos pedidos de todas las provincias, queremos que los datasets sean lo más homogéneos posibles.

Por otra parte, la columna 'FECHA NACIMIENTO' nos quedamos solo con el año, eliminando la hora. Por defecto, Excel pone el CP como float, asi que lo pasamos a entero.

Si existe en el dataframe, como en este caso, aquellos que no tienen registro de 'TIPO PRÓTESIS' hay que examinar el motivo. Vamos a ver aquellos registros distintos que tienen todas las columnas. Vemos que en la columna 'TIPO PRÓTESIS' no hay ninguna salida que sea "NO TIENE" y preguntamos a FASPAS, tras su aprobación, procedemos a sustituir los valores nulos por 'NO TIENE'.

Se observa que la 'MODALIDAD COMUNICATIVA' no da ninguna información, pues todos los registros son "ORAL", sim embargo, la dejamos por si acaso en un futuro los registros cambiaran.

Se disminuye la complejidad de los datos de manera que solo nos quedamos con la categoría de 'AUD O PRÓTESIS' que indica si lleva o no alguno de los dispotivos posibles (¿Lleva implante de tronco cerebral/osteointegrado/de oído medio?) con independencia de cuál de estos sea.

In [ ]:
# Creación de columna 'AUD O PRÓTESIS' en función del dataset de entrada
def determinar_si_o_no(row):
    # Lista de las columnas a verificar
    columns_to_meet = ['?LLEVA I.C.?', '?LLEVA IMPLANTE DE TRONCO CEREBRAL?', '?LLEVA IMPLANTE OSTEOINTEGRADO?',
                       '?LLEVA IMPLANTE DE OIDO MEDIO?', 'AUDIFONO']
    # Itera sobre cada columna especificada
    for col in columns_to_meet:
        # Verifica si el valor es NaN, "No" o "no tiene"
        if pd.isna(row[col]) or row[col] == 'NO' or row[col] == 'NO TIENE':
            return 'NO'
    # Si ninguna columna cumple la condición anterior, devuelve 'Sí'
    return 'SI'

def other_columns(df):
  if 'TIPO PROTESIS' in df.columns:
    df['TIPO PROTESIS'] = df['TIPO PROTESIS'].fillna('NO TIENE')
    df['AUD O PROTESIS'] = np.where(df['TIPO PROTESIS'] == 'NO TIENE', 'NO', 'SI')
    df.drop('TIPO PROTESIS', axis=1, inplace=True)

  # elif ('¿LLEVA AUDIFONO?' and '¿LLEVA I.C.?' and '¿LLEVA IMPLANTE DE TRONCO CEREBRAL?' and '¿LLEVA IMPLANTE OSTEOINTEGRADO?' and '¿LLEVA IMPLANTE DE OIDO MEDIO?') in df.columns:
  elif '?LLEVA I.C.?' in df.columns:
    for row in df:
      df['AUD O PROTESIS'] = df.apply(determinar_si_o_no, axis=1)

  # Creación de columna 'AUD O PRÓTESIS' y elimino las demás
  columns_to_drop =['?LLEVA AUDIFONO?' ,'?LLEVA I.C.?', '?LLEVA IMPLANTE DE TRONCO CEREBRAL?',
                    '?LLEVA IMPLANTE OSTEOINTEGRADO?', '?LLEVA IMPLANTE DE OIDO MEDIO?']
  df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)

  if 'GENERO' in df.columns:
    mujer = ['MUJER', 'HEMBRA', 'FEMENINO', 'FEM', 'F']
    hombre = ['HOMBRE', 'MACHO', 'MASCULINO', 'MASC', 'MAS']

    for m in mujer:
      df['GENERO'] = df['GENERO'].replace(m, 'M')

    for h in hombre:
      df['GENERO'] = df['GENERO'].replace(h, 'H')
  return df

Para contemplar que otros datos que se ingresen sean distintos, vamos a jugar con eliminar aquellas columnas que no sean esenciales para el estudio territorial, sino que nos den información más sociológica y sirvan para un estudio social; siempre y cuando el porcentaje de valores nulos sea mayor o igual que 1/3 del total de registros.

##**OJOOO AÑO O ANO DE ATENCIÓN, REVISAR**

In [ ]:
def delete_not_important_columns(df):
  # Añado 'AÑO ATENCIÓN' para SAAF
  needed_columns = ['AÑO ATENCION', 'FECHA NACIMIENTO', 'LOCALIDAD', 'CP', 'YEAR NACIMIENTO']

  # Creación de la lista de columnas consideradas para posible eliminación
  social_columns = [col for col in df.columns if col not in needed_columns]

  # Porcentaje máximo de valores nulos permitido
  max_percentage = 1/3

  # Identifico columnas para eliminar
  del_columns = []
  for col in social_columns:
      if df[col].isnull().sum() / len(df) >= max_percentage:
          del_columns.append(col)

  # Eliminar las columnas identificadas
  df.drop(columns=del_columns, inplace=True)
  return df

Además, pondremos todos los registros de MUJER como M y HOMBRE como H. Esto lo realizaremos con todos los dataframes. Contemplamos que los registros que llegan pueden tener otros nombres en este campo.

In [ ]:
def left_columns_norm(df):
  if 'TIPO SORDERA' in df.columns:
    # Lista con las posibles salidas permitidas
    strings_to_keep = ['NEUROSENSORIAL', 'CONDUCTIVA', 'MIXTA', 'TEL']

    # Sustituyo los registros distintos por la palabra 'NO REGISTRADO' (VER SI SE CAMBIA ESTO)
    df.loc[~df['TIPO SORDERA'].isin(strings_to_keep), 'TIPO SORDERA'] = 'NO REGISTRADO'

  if 'MOMENTO APARICION' in df.columns:
    strings_to_keep = ['PERILOCUTIVA', 'POSTLOCUTIVA', 'PRELOCUTIVA']

    # Sustituyo los registros distintos por la palabra 'NO REGISTRADO' (VER SI SE CAMBIA ESTO)
    df.loc[~df['MOMENTO APARICION'].isin(strings_to_keep), 'MOMENTO APARICION'] = 'NO REGISTRADO'

  if 'MODALIDAD COMUNICATIVA' in df.columns:
    strings_to_keep = ['SIGNO', 'LSE','SIGNOS', 'ORAL', 'BIMODAL', 'L.S.E.', 'L.S.E', 'SIGNOS NATURALES',
                       'ORAL,SIGNOS NATURALES', 'ORAL, SIGNOS NATURALES']

    # Sustituyo los registros distintos por la palabra 'NO REGISTRADO' (VER SI SE CAMBIA ESTO)
    df.loc[~df['MODALIDAD COMUNICATIVA'].isin(strings_to_keep), 'MODALIDAD COMUNICATIVA'] = 'NO REGISTRADO'

    # Normalizo todas las formas de llamar a las distintas modalidades comunicativas
    ORAL = ['ORAL', 'ORAL,SIGNOS NATURALES', 'ORAL, SIGNOS NATURALES']
    LSE = ['LSE', 'L.S.E.','L.S.E']
    SIGNOS = ['SIGNO', 'SIGNOS', 'SIGNOS NATURALES']

    for o in ORAL:
      df['MODALIDAD COMUNICATIVA'] = df['MODALIDAD COMUNICATIVA'].replace(o, 'ORAL')

    for l in LSE:
      df['MODALIDAD COMUNICATIVA'] = df['MODALIDAD COMUNICATIVA'].replace(l, 'LSE')

    for s in SIGNOS:
      df['MODALIDAD COMUNICATIVA'] = df['MODALIDAD COMUNICATIVA'].replace(s, 'SIGNOS')

  if 'GRADO PERDIDA' in df.columns:
    df['GRADO PERDIDA'] = df['GRADO PERDIDA'].astype(str)

    # Selecciono solo las 3 primeras letras (quitando la parte de DB si la tuviera)
    df['GRADO PERDIDA'] = df['GRADO PERDIDA'].str.slice(0, 3)

    strings_to_keep = ['DAP', 'DAM', 'DAS', 'DAL']

    df.loc[~df['GRADO PERDIDA'].isin(strings_to_keep), 'GRADO PERDIDA'] = np.nan

  if 'GRADO DISCAPACIDAD' in df.columns:
    df['GRADO DISCAPACIDAD'] = df['GRADO DISCAPACIDAD'].apply(lambda x: x / 100 if x > 1 else x)
    df['GRADO DISCAPACIDAD'] = df['GRADO DISCAPACIDAD'].fillna(0)
  return df

Aquí podemos ver la información del df, cuáles son las columnas que se tendrán en cuenta para este caso concreto y los diferentes datos que pueden tomar. Observamos que los únicos datos que pueden tener valores nulos son CP y LOCALIDAD, pero serán en registros distintos.

Se crea un filtro booleano que coge aquellos registros con ambos campos nulos y se lo aplicamos al dataframe. Ya que aquellos registros sin geolocalización no nos sirven para el objetivo principal que es saber el impacto que se tiene en las diferentes zonas para desarrollar un plan de ampliación territorial estratégico basado en el estudio estadístico.

In [ ]:
# Suponiendo que 'dict_of_dfs' es tu diccionario de DataFrames
for df_key, df in dic_dataframes.items():
    dic_dataframes[df_key] = transform_df(df)
    dic_dataframes[df_key] = mapping_names(df)
    dic_dataframes[df_key] = kept_columns(df)
    dic_dataframes[df_key] = CP_year(df)
    dic_dataframes[df_key] = other_columns(df)
    dic_dataframes[df_key] = delete_rows(df)

In [ ]:
CU_SAAF = dic_dataframes.get("CU_SAAF.xlsx")
TO_SAAF = dic_dataframes.get("TO_SAAF.xlsx")
AB_SAAF = dic_dataframes.get("AB_SAAF.xlsx")
GU_SAAF = dic_dataframes.get("GU_SAAF.xlsx")
CR_SAAF = dic_dataframes.get("CR_SAAF.xlsx")

In [ ]:
# # CÓDIGO DE USO PARA EL CURRO
# CU_SAAF = transform_df(CU_SAAF)
# CU_SAAF = mapping_names(CU_SAAF)
# CU_SAAF = kept_columns(CU_SAAF)
# CU_SAAF = CP_year(CU_SAAF)
# CU_SAAF = other_columns(CU_SAAF)
# CU_SAAF = delete_rows(CU_SAAF)

In [ ]:
# # CÓDIGO DE USO PARA EL CURRO
# GU_SAAF = transform_df(GU_SAAF)
# GU_SAAF = mapping_names(GU_SAAF)
# GU_SAAF = kept_columns(GU_SAAF)
# GU_SAAF = CP_year(GU_SAAF)
# GU_SAAF = other_columns(GU_SAAF)
# GU_SAAF = delete_rows(GU_SAAF)

In [ ]:
# # CÓDIGO DE USO PARA EL CURRO
# TO_SAAF = transform_df(TO_SAAF)
# TO_SAAF = mapping_names(TO_SAAF)
# TO_SAAF = kept_columns(TO_SAAF)
# TO_SAAF = CP_year(TO_SAAF)
# TO_SAAF = other_columns(TO_SAAF)
# TO_SAAF = delete_rows(TO_SAAF)

In [ ]:
# # CÓDIGO DE USO PARA EL CURRO
# AB_SAAF = transform_df(AB_SAAF)
# AB_SAAF = mapping_names(AB_SAAF)
# AB_SAAF = kept_columns(AB_SAAF)
# AB_SAAF = CP_year(AB_SAAF)
# AB_SAAF = other_columns(AB_SAAF)
# AB_SAAF = left_columns_norm(AB_SAAF)
# AB_SAAF = delete_rows(AB_SAAF)

In [ ]:
# # CÓDIGO DE USO PARA EL CURRO
# CR_SAAF = transform_df(CR_SAAF)
# CR_SAAF = mapping_names(CR_SAAF)
# CR_SAAF = kept_columns(CR_SAAF)
# CR_SAAF = CP_year(CR_SAAF)
# CR_SAAF = other_columns(CR_SAAF)

In [ ]:
CLM_SAAF = pd.concat([CU_SAAF, CR_SAAF, GU_SAAF, TO_SAAF, AB_SAAF])

In [ ]:
CLM_SAAF = delete_not_important_columns(CLM_SAAF)
CLM_SAAF = left_columns_norm(CLM_SAAF)
CLM_SAAF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 767 entries, 0 to 211
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   FECHA NACIMIENTO  767 non-null    datetime64[ns]
 1   LOCALIDAD         636 non-null    object        
 2   CP                545 non-null    Int64         
 3   TIPO SORDERA      767 non-null    object        
 4   YEAR NACIMIENTO   767 non-null    int32         
 5   AUD O PROTESIS    541 non-null    object        
 6   FECHA ALTA        550 non-null    datetime64[ns]
dtypes: Int64(1), datetime64[ns](2), int32(1), object(3)
memory usage: 45.7+ KB


In [ ]:
# ESTO LO HAGO DESPUÉS DE SACAR EL DOC PARA CLM ENTERO, ASÍ NO QUITO NINGÚN DATO PARA EL INFORME DE COMUNIDAD AUTÓNOMA
AB_SAAF = delete_not_important_columns(AB_SAAF)
AB_SAAF = left_columns_norm(AB_SAAF)

CU_SAAF = delete_not_important_columns(CU_SAAF)
CU_SAAF = left_columns_norm(CU_SAAF)

CR_SAAF = delete_not_important_columns(CR_SAAF)
CR_SAAF = left_columns_norm(CR_SAAF)

GU_SAAF = delete_not_important_columns(GU_SAAF)
GU_SAAF = left_columns_norm(GU_SAAF)

TO_SAAF = delete_not_important_columns(TO_SAAF)
TO_SAAF = left_columns_norm(TO_SAAF)

###Salvaguardar los archivos
Se guardan los archivos en la carpeta de las provincias correspondientes y en la de CLM.

In [ ]:
# List of DataFrames
dfs = [AB_SAAF, CR_SAAF, CU_SAAF, GU_SAAF, TO_SAAF, CLM_SAAF]

# Corresponding folder paths on Google Drive
folder_paths = [
    '/content/drive/My Drive/PFG_FASPAS/AB',
    '/content/drive/My Drive/PFG_FASPAS/CR',
    '/content/drive/My Drive/PFG_FASPAS/CU',
    '/content/drive/My Drive/PFG_FASPAS/GU',
    '/content/drive/My Drive/PFG_FASPAS/TO',
    '/content/drive/My Drive/PFG_FASPAS/CLM'
]

# Corresponding file names
file_names = ['AB_SAAF_limpio.xlsx', 'CR_SAAF_limpio.xlsx', 'CU_SAAF_limpio.xlsx', 'GU_SAAF_limpio.xlsx', 'TO_SAAF_limpio.xlsx', 'CLM_SAAF_limpio.xlsx']

In [ ]:
# Iterate over the DataFrames, folder paths, and file names
for df, folder_path, file_name in zip(dfs, folder_paths, file_names):
    # Define the complete file path
    file_path = f"{folder_path}/{file_name}"

    # Save the DataFrame as an Excel file
    df.to_excel(file_path, index=False)

print("All DataFrames have been exported successfully.")

All DataFrames have been exported successfully.
